Extracting all text from website

In [22]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Function to scrape text from a given URL
def scrape_text_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an error for bad response status
        # Determine the encoding of the response
        encoding = response.encoding if 'charset' in response.headers.get('content-type', '').lower() else None
        soup = BeautifulSoup(response.content, 'html.parser', from_encoding=encoding)
        # Extract text from the webpage and strip whitespaces
        text = soup.get_text().strip()
        # Join the words into one string
        text = ' '.join(text.split())
        return text
    except requests.RequestException as e:
        print(f"Error scraping URL '{url}': {e}")
        return "Link does not work"

# File paths
input_file_path = "/Users/ladderleap/code/yevkim/schemesv2/raw_data/schemes-updated-fine-tuned.csv"
output_file_path = "/Users/ladderleap/code/yevkim/schemesv2/raw_data/schemes-updated-with-text.csv"

# Read the CSV file into a DataFrame
schemes_updated_df = pd.read_csv(input_file_path, header=1)

# Extract text from each link in the 'Link' column and store it in a new column
schemes_updated_df['Scraped Text'] = schemes_updated_df['Link'].apply(lambda x: scrape_text_from_url(x))

# Save the updated DataFrame to a new CSV file with escapechar set to '\\'
schemes_updated_df.to_csv(output_file_path, index=False, escapechar='\\')

print(f"Updated DataFrame saved to: {output_file_path}")

Error scraping URL 'https://365cps.org.sg/portfolio/financial-support/': 403 Client Error: Forbidden for url: https://365cps.org.sg/portfolio/financial-support/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Error scraping URL 'https://www.aic.sg/financial-assistance/pioneer-generation-disability-assistance-scheme': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/pioneer-generation-disability-assistance-scheme
Error scraping URL 'https://www.aic.sg/financial-assistance/careshield-life': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/careshield-life
Error scraping URL 'https://www.aic.sg/financial-assistance/medifund': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/medifund
Error scraping URL 'https://aic.sg/financial-assistance/caregivers-training-grant': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/caregivers-training-grant
Error scraping URL 'https://www.aic.sg/financial-assistance/foreign-domestic-worker-levy-concession': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/foreign-domestic-worker-levy-concession
Error scraping URL 'https://www

In [23]:
schemes_updated_df

,Scheme,Description,Agency,Image,Link,Who's it for,What it gives,Scheme Type,search_booster(WL),Final_model_corpus,Eligibility,Unnamed: 11,Scraped Text
0,Various services,Information relating to social service and ref...,@27 Family Service Centre,https://chidnast.sirv.com/SchemesSG/27fsc.jpg,https://www.27fsc.org.sg/,Families,"Counselling,Casework","Low Income,Family","social service, individuals, families, casewor...",NaN,NaN,NaN,@27 Family Service Centre (@27FSC) – Just anot...
1,Financial Assistance,Cancer treatment can place a heavy financial b...,365 Cancer Prevention Society (365CPS),https://chidnast.sirv.com/SchemesSG/365cps.jpg,https://365cps.org.sg/portfolio/financial-supp...,Cancer patients,Financial assistance for cancer treatment,"Low Income,Healthcare","See doctor, cancer, oncology",NaN,NaN,NaN,Link does not work
2,Food Assistance,A self setup group which distributes meal box ...,A Packet of Rice,https://chidnast.sirv.com/SchemesSG/apacketofr...,https://www.facebook.com/APacketOfRice/,"Low income,Need food support",Food,"Low Income,Food","Needs help to get food, meal, hungry, have not...",NaN,NaN,NaN,A Packet of Rice
3,Rehabilitation Centre and Respite Centre,"Provides Occupational Therapy, Physiotherapy a...",Abilities Beyond Limitations and Expectations ...,https://chidnast.sirv.com/SchemesSG/able.jpg,https://www.able-sg.org/,"PWD,Elderly,Caregivers","Occupational therapy,Physiotherapy,Educational...","PWD,Family,Transport,Healthcare,Mental Health","rehabilitation, occupational therapy, physioth...",NaN,NaN,NaN,Abilities Beyond Limitations and Expectations ...
4,Various Services,We prepare educational programs (IT Training) ...,Acronis Foundation,https://chidnast.sirv.com/SchemesSG/acronis.jpg,https://acronis.org/rehabilitation-of-inmates/,Ex-offenders,"Vocational training,Employment assistance","Ex-offender,Employment","ex-convict need job, ex-convict skills trainin...",NaN,NaN,NaN,IT Skills Program for ex-offenders – Acronis C...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
324,Migrant Domestic Worker Levy Concession,Lets families pay the Migrant Domestic Worker ...,Agency for Integrated Care (AIC),https://chidnast.sirv.com/SchemesSG/aic.jpg,https://www.aic.sg/financial-assistance/migran...,Foreign Domestic Workers/maids,Financial assistance,"Foreign Domestic Workers/maids,Women","Hire helper for caregiving, take care of elder...",NaN,NaN,NaN,Link does not work
325,Mid-Career Pathways Programme (MCPP),"A full-time attachment programme for mature, m...",Workforce Singapore (WSG),https://chidnast.sirv.com/SchemesSG/wsg.jpg,https://www.wsg.gov.sg/home/individuals/attach...,"Unemployed,Retrenched,In need of job,Upskilling","Employment assistance,Vocational training","Work,Employment","Mid-career, change job, unemployment, upskilli...",NaN,NaN,NaN,Mid-Career Individuals | Mid-Career Pathways P...
326,SkillsFuture Career Transition Programme (SCTP),Industry-oriented training programmes to suppo...,Skillsfuture SG,NaN,https://www.skillsfuture.gov.sg/initiatives/mi...,"Unemployed,Retrenched,In need of job,Upskilling","Employment assistance,Vocational training","Work,Employment","Mid-career, change job, unemployment, upskilli...",NaN,NaN,NaN,SSG | SkillsFuture Career Transition Programme...
327,ITE Monthly Financial Assistance Scheme (MFAS),Provides a monthly allowance to full-time Sing...,Institute of Technical Education (ITE),NaN,https://www.ite.edu.sg/life-at-ite/student-ser...,"Low income,Facing financial hardship",Financial assistance,Low Income,"Need money for daily needs, low income, poor",NaN,NaN,NaN,Financial Assistance Student Login Staff Login...


In [24]:
num_failed_links = (schemes_updated_df['Scraped Text'] == 'Link does not work').sum()

print(f"Number of rows with 'Link does not work': {num_failed_links}")

Number of rows with 'Link does not work': 72


In [25]:
# Create empty lists to store information about failed links
failed_links = []
failed_schemes = []
failed_descriptions = []
failed_errors = []

# Iterate through the DataFrame and collect information about failed links
for index, row in schemes_updated_df.iterrows():
    try:
        scraped_text = scrape_text_from_url(row['Link'])
        if scraped_text == "Link does not work":
            raise Exception("Link does not work")
        schemes_updated_df.at[index, 'Scraped Text'] = scraped_text
    except requests.RequestException as e:
        error_message = f"Error scraping URL '{row['Link']}': {str(e)}"
        failed_links.append(row['Link'])
        failed_schemes.append(row['Scheme'])
        failed_descriptions.append(row['Description'])
        failed_errors.append(error_message)
        schemes_updated_df.at[index, 'Error'] = error_message
    except Exception as e:
        error_message = f"Error scraping URL '{row['Link']}': {str(e)}"
        failed_links.append(row['Link'])
        failed_schemes.append(row['Scheme'])
        failed_descriptions.append(row['Description'])
        failed_errors.append(error_message)
        schemes_updated_df.at[index, 'Error'] = error_message

# Create DataFrame for failed links
failed_links_df = pd.DataFrame({
    'Link': failed_links,
    'Scheme': failed_schemes,
    'Description': failed_descriptions,
    'Error': failed_errors
})

# Save the DataFrame for failed links to a CSV file
failed_links_output_path = "/Users/ladderleap/code/yevkim/schemesv2/raw_data/failed_links.csv"
failed_links_df.to_csv(failed_links_output_path, index=False)
print(f"Failed links saved to: {failed_links_output_path}")


Error scraping URL 'https://365cps.org.sg/portfolio/financial-support/': 403 Client Error: Forbidden for url: https://365cps.org.sg/portfolio/financial-support/


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Error scraping URL 'https://www.aic.sg/financial-assistance/pioneer-generation-disability-assistance-scheme': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/pioneer-generation-disability-assistance-scheme
Error scraping URL 'https://www.aic.sg/financial-assistance/careshield-life': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/careshield-life
Error scraping URL 'https://www.aic.sg/financial-assistance/medifund': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/medifund
Error scraping URL 'https://aic.sg/financial-assistance/caregivers-training-grant': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/caregivers-training-grant
Error scraping URL 'https://www.aic.sg/financial-assistance/foreign-domestic-worker-levy-concession': 403 Client Error: Forbidden for url: https://www.aic.sg/financial-assistance/foreign-domestic-worker-levy-concession
Error scraping URL 'https://www

In [26]:
#failed links due to restricted access to server(error 403)
failed_links_df

,Link,Scheme,Description,Error
0,https://365cps.org.sg/portfolio/financial-supp...,Financial Assistance,Cancer treatment can place a heavy financial b...,Error scraping URL 'https://365cps.org.sg/port...
1,https://www.aic.sg/financial-assistance/pionee...,Pioneer Generation Disability Assistance Schem...,The Pioneer Generation Disability Assistance S...,Error scraping URL 'https://www.aic.sg/financi...
2,https://www.aic.sg/financial-assistance/caresh...,CareShield Life,CareShield Life is a long-term care insurance ...,Error scraping URL 'https://www.aic.sg/financi...
3,https://www.aic.sg/financial-assistance/medifund,Medifund,Medifund helps needy Singapore citizens who ca...,Error scraping URL 'https://www.aic.sg/financi...
4,https://aic.sg/financial-assistance/caregivers...,Caregivers Training Grant (CTG),Helps caregivers defray the cost of attending ...,Error scraping URL 'https://aic.sg/financial-a...
...,...,...,...,...
68,https://www.hdb.gov.sg/cs/infoweb/residential/...,Parenthood Provisional Housing Scheme (PPHS),Provides temporary housing for families as the...,Error scraping URL 'https://www.hdb.gov.sg/cs/...
69,https://www.aic.sg/financial-assistance/senior...,Seniors' Mobility and Enabling Fund,Provides subsidies to seniors for mobility and...,Error scraping URL 'https://www.aic.sg/financi...
70,https://silverjobs.sg/,Silverjobs Portal,A job portal for seniors to explore work in va...,Error scraping URL 'https://silverjobs.sg/': L...
71,https://www.hdb.gov.sg/residential/buying-a-fl...,Community Care Apartments,Senior-friendly housing integrated with care s...,Error scraping URL 'https://www.hdb.gov.sg/res...
